In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Load files
institutional
File name summary
sf3 quarterly holding enttries by stock and institutional investor
sf3a: summary of sf3 by ticker
sf3b: summary of sf3 by investor
sf1: quarterly fundamentals for the stock

In [7]:
# Read SF3 table from CSV file
sf3 = pd.read_csv('data/SHARADAR_holdings.csv')

# Read SF3A table from CSV file
sf3a = pd.read_csv('data/SHARADAR_SF3A_apr5.csv')

# Read SF3B table from CSV file
sf3b = pd.read_csv('data/SHARADAR_SF3B_apr5.csv')

sf1_file_path = '/Users/stirlitz/Downloads/data/SHARADAR_SF1_apr5.csv'

sf1 = pd.read_csv(sf1_file_path)

print(sf1.head())

  ticker dimension calendardate     datekey reportperiod lastupdated   
0     HP       ARQ   1993-12-31  1994-02-14   1993-12-31  2023-01-30  \
1     HP       ARQ   1994-03-31  1994-05-13   1994-03-31  2023-01-30   
2     HP       ARQ   1994-06-30  1994-08-12   1994-06-30  2023-01-30   
3     HP       ARQ   1994-09-30  1994-12-22   1994-09-30  2023-01-30   
4     HP       ARQ   1994-12-31  1995-02-14   1994-12-31  2023-01-30   

       accoci       assets  assetsavg      assetsc  ...   sharesbas   
0         0.0  615990000.0        NaN  155036000.0  ...  98669928.0  \
1         0.0  625211000.0        NaN  162414000.0  ...  98669928.0   
2         0.0  621476000.0        NaN  135679000.0  ...  98809664.0   
3         0.0  624827000.0        NaN  122939000.0  ...  98874640.0   
4  30700000.0  676284000.0        NaN  111225000.0  ...  98874640.0   

     shareswa  shareswadil    sps    tangibles  taxassets     taxexp   
0  97473928.0   97473928.0  0.843  615990000.0        0.0  4183000.0

Check if I understand 

In [5]:
# Group the SF3A table by 'calendardate' and calculate the sum of 'percentoftotal'
percent_total_by_date = sf3a.groupby('calendardate')['percentoftotal'].sum().reset_index()

# Print the results
print("Date-wise sum of 'percentoftotal':")
print(percent_total_by_date)

Date-wise sum of 'percentoftotal':
   calendardate  percentoftotal
0    2013-06-30          99.659
1    2013-09-30          99.654
2    2013-12-31          99.668
3    2014-03-31          99.643
4    2014-06-30          99.636
5    2014-09-30          99.643
6    2014-12-31          99.630
7    2015-03-31          99.629
8    2015-06-30          99.673
9    2015-09-30          99.679
10   2015-12-31          99.693
11   2016-03-31          99.652
12   2016-06-30          99.638
13   2016-09-30          99.640
14   2016-12-31          99.670
15   2017-03-31          99.667
16   2017-06-30          99.670
17   2017-09-30          99.625
18   2017-12-31          99.639
19   2018-03-31          99.626
20   2018-06-30          99.660
21   2018-09-30          99.667
22   2018-12-31          99.625
23   2019-03-31          99.652
24   2019-06-30          99.651
25   2019-09-30          99.642
26   2019-12-31          99.628
27   2020-03-31          99.643
28   2020-06-30          99.652
29   

In [16]:
# for fundamentals data (sf1) I want to only use rows where dimension is ARQ (as reported quarterly)
# Merge the necessary columns from sf1 with the investor summary table (sf3a)

merged_df = sf3a.merge(sf1[['ticker', 'calendardate', 'price']], on=['ticker', 'calendardate'], how='left')

# Calculate the product of shares held (shrunits) and share price
merged_df['calculated_value'] = merged_df['shrunits'] * merged_df['price']

# Compare the calculated value to the value held in the investor summary table (shrvalue)
merged_df['values_match'] = merged_df['calculated_value'].round(2) == merged_df['shrvalue'].round(2)

# Print the result
print(merged_df[['ticker', 'calendardate', 'shrunits', 'price', 'shrvalue', 'calculated_value', 'values_match']].head())

# Check if all the values match
all_values_match = merged_df['values_match'].all()
print("\nDo all the calculated values match the values in the investor summary table?", all_values_match)


  ticker calendardate   shrunits  price     shrvalue  calculated_value   
0   AAPL   2023-03-31  1740861.0    NaN  287039034.0               NaN  \
1   MSFT   2023-03-31   986053.0    NaN  284274952.0               NaN   
2    VOO   2023-03-31        0.0    NaN          0.0               NaN   
3    PRK   2023-03-31  1414437.0    NaN  167709802.0               NaN   
4    JPM   2023-03-31  1067924.0    NaN  139151804.0               NaN   

   values_match  
0         False  
1         False  
2         False  
3         False  
4         False  

Do all the calculated values match the values in the investor summary table? False


In [26]:
sf1[['ticker', 'calendardate', 'price']]
merged_df[['ticker', 'calendardate', 'price']]

sf1[sf1['ticker'] == 'AAPL']
sf3a[sf3a['ticker'] == 'AAPL']

,calendardate,ticker,name,shrholders,cllholders,putholders,wntholders,dbtholders,prfholders,fndholders,...,shrvalue,cllvalue,putvalue,wntvalue,dbtvalue,prfvalue,fndvalue,undvalue,totalvalue,percentoftotal
0,2023-03-31,AAPL,APPLE INC,15,0,0,0,0,0,0,...,2.870390e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,2.870390e+08,3.120
1432,2022-12-31,AAPL,APPLE INC,4354,131,166,0,1,0,0,...,1.221911e+12,3.065748e+10,3.108592e+10,0.0,4868.0,0.0,0.0,0.0,1.283654e+12,3.155
14429,2022-09-30,AAPL,APPLE INC,4207,127,155,0,2,0,0,...,1.280001e+12,3.082557e+10,3.668594e+10,0.0,77000.0,0.0,0.0,0.0,1.347513e+12,3.675
27549,2022-06-30,AAPL,APPLE INC,4208,131,153,0,1,0,0,...,1.269353e+12,2.583685e+10,3.355410e+10,0.0,53000.0,0.0,0.0,0.0,1.328744e+12,3.458
40759,2022-03-31,AAPL,APPLE INC,4260,130,139,0,1,0,0,...,1.626806e+12,3.592070e+10,4.325106e+10,0.0,9000.0,0.0,0.0,0.0,1.705978e+12,3.692
53956,2021-12-31,AAPL,APPLE INC,4279,141,125,0,1,0,0,...,1.682472e+12,3.936796e+10,5.265592e+10,0.0,597000.0,0.0,0.0,0.0,1.774496e+12,3.562
67144,2021-09-30,AAPL,APPLE INC,3762,146,126,0,1,0,0,...,1.335875e+12,3.312011e+10,4.244416e+10,0.0,603000.0,0.0,0.0,0.0,1.411440e+12,3.056
79875,2021-06-30,AAPL,APPLE INC,3783,154,120,0,1,0,0,...,1.302437e+12,3.301631e+10,3.898035e+10,0.0,607000.0,0.0,0.0,0.0,1.374434e+12,2.979
93086,2021-03-31,AAPL,APPLE INC,3765,146,116,0,2,0,0,...,1.168278e+12,3.122627e+10,3.728108e+10,0.0,1176000.0,0.0,0.0,0.0,1.236786e+12,2.909
106009,2020-12-31,AAPL,APPLE INC,3769,148,121,0,2,0,0,...,1.309815e+12,4.544115e+10,4.616889e+10,0.0,1194000.0,0.0,0.0,0.0,1.401426e+12,3.472
